# DDSIM vs Qiskit Aer

In [ ]:
import time
import random

import matplotlib.pyplot as plt
from mqt import ddsim
import numpy as np
import qiskit
import qiskit_aer

## One case

### Create the target circuits

In [ ]:
num_state_qubits = 13
circuit = qiskit.circuit.library.DraperQFTAdder(num_state_qubits)

# ==========
a = 7
b = 5
# ==========
a_bin = bin(a)[2:]
b_bin = bin(b)[2:]
out_bin = bin(a + b)[2:]
print(f"a + b = {a} + {b} = {int(out_bin, 2)}")

for index, letter in enumerate(a_bin[::-1]):
    if letter == "1":
        circuit.compose(qiskit.circuit.library.XGate(), [index], front=True, inplace=True)
for index, letter in enumerate(b_bin[::-1], num_state_qubits):
    if letter == "1":
        circuit.compose(qiskit.circuit.library.XGate(), [index], front=True, inplace=True)

# circuit.draw()

### Define utils

In [ ]:
SEED = 901

In [ ]:
def convert_counts(original_counts):
    return {int(value, 16): original_counts[value] for value in original_counts}

In [ ]:
def get_transpiled(backend):
    pass_manager = qiskit.transpiler.generate_preset_pass_manager(
        optimization_level=3,
        backend=backend,
        seed_transpiler=SEED,
    )

    trainspiled_circuit = pass_manager.run(circuit)  # Use the circuit defined above

    trainspiled_circuit.add_register(qiskit.ClassicalRegister(circuit.num_qubits - circuit.num_state_qubits))
    trainspiled_circuit.measure(trainspiled_circuit.qregs[-1], trainspiled_circuit.cregs[0])
    
    return trainspiled_circuit

In [ ]:
def run(backend):
    assert SEED == backend.options.seed_simulator
    transpiled_circuit = get_transpiled(backend)
    
    job = backend.run(transpiled_circuit, shots=8192)

    return job

### Check the backends

In [ ]:
ddsim.DDSIMProvider().backends()

In [ ]:
qiskit_aer.AerProvider().backends()

### QASM

#### AerSimulator

In [ ]:
aer_qasm = qiskit_aer.AerSimulator(seed_simulator=SEED)
aer_qasm_job = run(aer_qasm)

In [ ]:
print(aer_qasm_job.status())
print(convert_counts(aer_qasm_job.result().results[0].data.counts))
aer_qasm_job.result().results[0].metadata["time_taken"]

#### Among DDSIM

##### `QasmSimulatorBackend`

In [ ]:
ddsim_qasm = ddsim.qasmsimulator.QasmSimulatorBackend()
ddsim_qasm.options.update_options(seed_simulator=SEED)

ddsim_qasm_job = run(ddsim_qasm)

In [ ]:
print(ddsim_qasm_job.status())
print(convert_counts(ddsim_qasm_job.result().results[0].data.counts))
ddsim_qasm_job.result().results[0].data.time_taken

##### `HybridQasmSimulatorBackend`
`MemoryError: std::bad_alloc` arises.

In [ ]:
# ddsim_hybrid_qasm = ddsim.hybridqasmsimulator.HybridQasmSimulatorBackend()
# ddsim_hybrid_qasm.options.update_options(seed_simulator=SEED)
# 1
# ddsim_hybrid_qasm_job = run(ddsim_hybrid_qasm)

In [ ]:
# print(convert_counts(ddsim_hybrid_qasm_job.result().results[0].data.counts))
# ddsim_qasm_job.result().results[0].data.time_taken

##### `PathQasmSimulatorBackend`
`MemoryError: std::bad_alloc` arises.

In [ ]:
# ddsim_path_qasm = ddsim.pathqasmsimulator.PathQasmSimulatorBackend()
# ddsim_path_qasm.options.update_options(seed_simulator=SEED)

# ddsim_path_qasm_job = run(ddsim_hybrid_qasm)

In [ ]:
# print(convert_counts(ddsim_path_qasm_job.result().results[0].data.counts))
# ddsim_path_qasm_job.result().results[0].data.time_taken

## Comprehensive Analysis

### Increase in the number of qubtis

In [ ]:
def get_circuit(num_state_qubits, a, b):
    circuit = qiskit.circuit.library.DraperQFTAdder(num_state_qubits, kind='half')

    a_bin = bin(a)[2:]
    b_bin = bin(b)[2:]
    out_bin = bin(a + b)[2:]
    # print(f"a + b = {a} + {b} = {int(out_bin, 2)}")
    
    for index, letter in enumerate(a_bin[::-1]):
        if letter == "1":
            circuit.compose(qiskit.circuit.library.XGate(), [index], front=True, inplace=True)
    for index, letter in enumerate(b_bin[::-1], num_state_qubits):
        if letter == "1":
            circuit.compose(qiskit.circuit.library.XGate(), [index], front=True, inplace=True)

    return circuit

In [ ]:
SEED = 901
SHOTS = 8192

def convert_counts(original_counts):
    return {int(value, 16): original_counts[value] for value in original_counts}

In [ ]:
aer_qasm = qiskit_aer.AerSimulator(seed_simulator=SEED)

ddsim_qasm = ddsim.qasmsimulator.QasmSimulatorBackend()
ddsim_qasm.options.update_options(seed_simulator=SEED)

In [ ]:
%%time
random.seed(SEED)

max_num_state_qubits = 15
num_trials = 10

aer_jobs = dict()
ddsim_jobs = dict()
for num_state_qubits in range(1, max_num_state_qubits+1):
    for _ in range(num_trials):
        max_number = (2 ** num_state_qubits) - 1
        a = random.randint(0, max_number)
        b = random.randint(0, max_number)
        
        # Get the target circuit.
        circuit = get_circuit(num_state_qubits, a, b)
        
        # Make the key for the dicts.
        result = a + b
        key = (circuit.num_qubits, num_trials, a, b, result)
    
        # Transpile the circuit for Aer.
        pass_manager = qiskit.transpiler.generate_preset_pass_manager(
            optimization_level=3,
            backend=aer_qasm,
            seed_transpiler=SEED,
        )
        trainspiled_circuit_for_aer = pass_manager.run(circuit)
        trainspiled_circuit_for_aer.add_register(
            qiskit.ClassicalRegister(circuit.num_qubits - circuit.num_state_qubits)
        )
        for c_index in range(trainspiled_circuit_for_aer.num_qubits - num_state_qubits):
            q_index = c_index + num_state_qubits
            trainspiled_circuit_for_aer.measure(q_index, c_index)
        # Run the circuit with Aer.
        job = aer_qasm.run(trainspiled_circuit_for_aer, shots=SHOTS)
        aer_jobs[key] = job
    
        # Transpile the circuit for DDSIM.
        pass_manager = qiskit.transpiler.generate_preset_pass_manager(
            optimization_level=3,
            backend=ddsim_qasm,
            seed_transpiler=SEED,
        )
        trainspiled_circuit_for_ddsim = pass_manager.run(circuit)
        trainspiled_circuit_for_ddsim.add_register(
            qiskit.ClassicalRegister(circuit.num_qubits - circuit.num_state_qubits)
        )
        for c_index in range(trainspiled_circuit_for_ddsim.num_qubits - num_state_qubits):
            q_index = c_index + num_state_qubits
            trainspiled_circuit_for_ddsim.measure(q_index, c_index)
        # Run the circuit with DDSIM.
        job = ddsim_qasm.run(trainspiled_circuit_for_ddsim, shots=SHOTS)
        ddsim_jobs[key] = job

In [ ]:
# Define functions to organise data.
def get_success_possibility(job, true_key_digit, shots=SHOTS):
    counts_digit = convert_counts(job.result().results[0].data.counts)
    return counts_digit[true_key_digit] / shots

def get_time_taken_from_aer(job):
    return job.result().results[0].metadata["time_taken"]

def get_time_taken_from_ddsim(job):
    return job.result().results[0].data.time_taken

In [ ]:
# Organise the data for Aer.
aer_results = dict()
for key, aer_job in aer_jobs.items():
    num_qubits = key[0]
    if num_qubits not in aer_results:
        aer_results[num_qubits] = {"success_possibilities": [], "time_taken": []}

    if not aer_job.result().results[0].success:
        continue
    true_key_digit = key[-1]
    aer_results[num_qubits]["success_possibilities"].append(get_success_possibility(aer_job, true_key_digit))
    aer_results[num_qubits]["time_taken"].append(get_time_taken_from_aer(aer_job))

aer_result = [
    (num_qubits,
     float(np.mean(aer_results[num_qubits]["success_possibilities"])),
     float(np.mean(aer_results[num_qubits]["time_taken"])),
    )
    for num_qubits in aer_results.keys()
]
aer_result

In [ ]:
# Organise the data for DDSIM.
ddsim_results = dict()
for key, ddsim_job in ddsim_jobs.items():
    num_qubits = key[0]
    if num_qubits not in ddsim_results:
        ddsim_results[num_qubits] = {"success_possibilities": [], "time_taken": []}

    if not ddsim_job.result().results[0].success:
        continue
    
    true_key_digit = key[-1]
    ddsim_results[num_qubits]["success_possibilities"].append(get_success_possibility(ddsim_job, true_key_digit))
    ddsim_results[num_qubits]["time_taken"].append(get_time_taken_from_ddsim(ddsim_job))

ddsim_result = [
    (num_qubits,
     float(np.mean(ddsim_results[num_qubits]["success_possibilities"])),
     float(np.mean(ddsim_results[num_qubits]["time_taken"])),
    )
    for num_qubits in ddsim_results.keys()
]
ddsim_result

In [ ]:
assert [num_qubits  for num_qubits, _, _ in aer_result] == [num_qubits  for num_qubits, _, _ in ddsim_result]
x = [num_qubits  for num_qubits, _, _ in aer_result]
success_possibilities_aer = [success_possibility  for _, success_possibility, _ in aer_result]
success_possibilities_ddsim = [success_possibility  for _, success_possibility, _ in ddsim_result]
times_taken_aer = [time_taken  for _, _, time_taken in aer_result]
times_taken_ddsim = [time_taken  for _, _, time_taken in ddsim_result]

fig, axes = plt.subplots(ncols=2, figsize=(10,4))

# Success probability
axes[0].scatter(x, success_possibilities_aer, color="blue", s=20, label="AerSimulator")
axes[0].scatter(x, success_possibilities_ddsim, color="red", s=5, label="DDSIMulator")
axes[0].set_xlabel("the number of qubits")
axes[0].set_ylabel("Avg. success probability")
axes[0].legend()

# Time taken
axes[1].scatter(x, times_taken_aer, color="blue", s=20, label="AerSimulator")
axes[1].scatter(x, times_taken_ddsim, color="red", s=5, label="DDSIMulator")
axes[1].set_xlabel("the number of qubits")
axes[1].set_ylabel("Avg. time to execute")
axes[1].legend()

plt.savefig("ddsim_vs_aer_qubits.png")

plt.show()

In [ ]:
%%time
random.seed(SEED)

additional_max_num_state_qubits = 42
num_trials = 10

additional_ddsim_jobs = dict()
for num_state_qubits in range(max_num_state_qubits+1, additional_max_num_state_qubits+1):
    for _ in range(num_trials):
        max_number = (2 ** num_state_qubits) - 1
        a = random.randint(0, max_number)
        b = random.randint(0, max_number)
        
        # Get the target circuit.
        circuit = get_circuit(num_state_qubits, a, b)
        
        # Make the key for the dicts.
        result = a + b
        key = (circuit.num_qubits, num_trials, a, b, result)

        # Transpile the circuit for DDSIM.
        pass_manager = qiskit.transpiler.generate_preset_pass_manager(
            optimization_level=3,
            backend=ddsim_qasm,
            seed_transpiler=SEED,
        )
        trainspiled_circuit_for_ddsim = pass_manager.run(circuit)
        trainspiled_circuit_for_ddsim.add_register(
            qiskit.ClassicalRegister(circuit.num_qubits - circuit.num_state_qubits)
        )
        for c_index in range(trainspiled_circuit_for_ddsim.num_qubits - num_state_qubits):
            q_index = c_index + num_state_qubits
            trainspiled_circuit_for_ddsim.measure(q_index, c_index)
        # Run the circuit with DDSIM.
        job = ddsim_qasm.run(trainspiled_circuit_for_ddsim, shots=SHOTS)
        additional_ddsim_jobs[key] = job

In [ ]:
# Organise the data for DDSIM.
additional_ddsim_results = dict()
for key, ddsim_job in additional_ddsim_jobs.items():
    num_qubits = key[0]
    if num_qubits not in additional_ddsim_results:
        additional_ddsim_results[num_qubits] = {"success_possibilities": [], "time_taken": []}

    if not ddsim_job.result().results[0].success:
        continue
    
    true_key_digit = key[-1]
    additional_ddsim_results[num_qubits]["success_possibilities"].append(get_success_possibility(ddsim_job, true_key_digit))
    additional_ddsim_results[num_qubits]["time_taken"].append(get_time_taken_from_ddsim(ddsim_job))

additional_ddsim_result = [
    (num_qubits,
     float(np.mean(additional_ddsim_results[num_qubits]["success_possibilities"])),
     float(np.mean(additional_ddsim_results[num_qubits]["time_taken"])),
    )
    for num_qubits in additional_ddsim_results.keys()
]
additional_ddsim_result

In [ ]:
x = [num_qubits for num_qubits, _, _ in ddsim_result]
additional_x = [num_qubits for num_qubits, _, _ in additional_ddsim_result]

success_possibilities_ddsim = [success_possibility  for _, success_possibility, _ in ddsim_result]
times_taken_ddsim = [time_taken  for _, _, time_taken in ddsim_result]

additional_success_possibilities_ddsim = [success_possibility  for _, success_possibility, _ in additional_ddsim_result]
additional_times_taken_ddsim = [time_taken  for _, _, time_taken in additional_ddsim_result]

fig, axes = plt.subplots(ncols=2, figsize=(10,4))

# Success probability
axes[0].scatter(x, success_possibilities_ddsim, color="red", s=5, label="DDSIMulator")
axes[0].scatter(additional_x, additional_success_possibilities_ddsim, color="red", s=5, label="DDSIMulator")
axes[0].set_xlabel("the number of qubits")
axes[0].set_ylabel("Avg. success probability")
axes[0].legend()

# Time taken
axes[1].scatter(x, times_taken_ddsim, color="red", s=5, label="DDSIMulator")
axes[1].scatter(additional_x, additional_times_taken_ddsim, color="red", s=5, label="DDSIMulator")
axes[1].set_xlabel("the number of qubits")
axes[1].set_ylabel("Avg. time to execute")
axes[1].legend()

plt.savefig("additional_ddsim_qubits.png")

plt.show()

### Increase in the number of adder

In [ ]:
def get_deep_circuit(num_state_qubits, a, b, depth):
    if depth <= 0:
        depth = 1

    circuit = qiskit.circuit.library.DraperQFTAdder(num_state_qubits, kind='half')

    a_bin = bin(a)[2:]
    b_bin = bin(b)[2:]
    out_bin = bin(a + b)[2:]
    # print(f"a + b = {a} + {b} = {int(out_bin, 2)}")
    
    for index, letter in enumerate(a_bin[::-1]):
        if letter == "1":
            circuit.compose(qiskit.circuit.library.XGate(), [index], front=True, inplace=True)
    for index, letter in enumerate(b_bin[::-1], num_state_qubits):
        if letter == "1":
            circuit.compose(qiskit.circuit.library.XGate(), [index], front=True, inplace=True)

    for _ in range(depth - 1):
        circuit.compose(qiskit.circuit.library.DraperQFTAdder(num_state_qubits, kind='half'), range(circuit.num_qubits), inplace=True)

    return circuit

In [ ]:
%%time
random.seed(SEED)

num_state_qubits = 10
max_depth = 20
num_trials = 10

aer_jobs = dict()
ddsim_jobs = dict()
for depth in range(1, max_depth + 1):
    for _ in range(num_trials):
        max_number = (2 ** num_state_qubits) - 1
        a = random.randint(0, max_number)
        b = random.randint(0, max_number)
        
        # Get the target circuit.
        circuit = get_deep_circuit(num_state_qubits, a, b, depth)
        
        # Make the key for the dicts.
        result = (a * depth + b) % (2 ** (num_state_qubits + 1))
        key = (depth, num_trials, a, b, result)
    
        # Transpile the circuit for Aer.
        pass_manager = qiskit.transpiler.generate_preset_pass_manager(
            optimization_level=3,
            backend=aer_qasm,
            seed_transpiler=SEED,
        )
        trainspiled_circuit_for_aer = pass_manager.run(circuit)
        trainspiled_circuit_for_aer.add_register(
            qiskit.ClassicalRegister(circuit.num_qubits - circuit.num_state_qubits)
        )
        for c_index in range(trainspiled_circuit_for_aer.num_qubits - num_state_qubits):
            q_index = c_index + num_state_qubits
            trainspiled_circuit_for_aer.measure(q_index, c_index)
        # Run the circuit with Aer.
        job = aer_qasm.run(trainspiled_circuit_for_aer, shots=SHOTS)
        aer_jobs[key] = job
    
        # Transpile the circuit for DDSIM.
        pass_manager = qiskit.transpiler.generate_preset_pass_manager(
            optimization_level=3,
            backend=ddsim_qasm,
            seed_transpiler=SEED,
        )
        trainspiled_circuit_for_ddsim = pass_manager.run(circuit)
        trainspiled_circuit_for_ddsim.add_register(
            qiskit.ClassicalRegister(circuit.num_qubits - circuit.num_state_qubits)
        )
        for c_index in range(trainspiled_circuit_for_ddsim.num_qubits - num_state_qubits):
            q_index = c_index + num_state_qubits
            trainspiled_circuit_for_ddsim.measure(q_index, c_index)
        # Run the circuit with DDSIM.
        job = ddsim_qasm.run(trainspiled_circuit_for_ddsim, shots=SHOTS)
        ddsim_jobs[key] = job

In [ ]:
# Organise the data for Aer.
aer_results = dict()
for key, aer_job in aer_jobs.items():
    num_qubits = key[0]
    if num_qubits not in aer_results:
        aer_results[num_qubits] = {"success_possibilities": [], "time_taken": []}

    if not aer_job.result().results[0].success:
        continue

    true_key_digit = key[-1]
    aer_results[num_qubits]["success_possibilities"].append(get_success_possibility(aer_job, true_key_digit))
    aer_results[num_qubits]["time_taken"].append(get_time_taken_from_aer(aer_job))

aer_result = [
    (num_qubits,
     float(np.mean(aer_results[num_qubits]["success_possibilities"])),
     float(np.mean(aer_results[num_qubits]["time_taken"])),
    )
    for num_qubits in aer_results.keys()
]
aer_result

In [ ]:
# Organise the data for DDSIM.
ddsim_results = dict()
for key, ddsim_job in ddsim_jobs.items():
    num_qubits = key[0]
    if num_qubits not in ddsim_results:
        ddsim_results[num_qubits] = {"success_possibilities": [], "time_taken": []}

    if not ddsim_job.result().results[0].success:
        continue
    
    true_key_digit = key[-1]
    ddsim_results[num_qubits]["success_possibilities"].append(get_success_possibility(ddsim_job, true_key_digit))
    ddsim_results[num_qubits]["time_taken"].append(get_time_taken_from_ddsim(ddsim_job))

ddsim_result = [
    (num_qubits,
     float(np.mean(ddsim_results[num_qubits]["success_possibilities"])),
     float(np.mean(ddsim_results[num_qubits]["time_taken"])),
    )
    for num_qubits in ddsim_results.keys()
]
ddsim_result

In [ ]:
assert [num_qubits  for num_qubits, _, _ in aer_result] == [num_qubits  for num_qubits, _, _ in ddsim_result]
x = [num_qubits  for num_qubits, _, _ in aer_result]
success_possibilities_aer = [success_possibility  for _, success_possibility, _ in aer_result]
success_possibilities_ddsim = [success_possibility  for _, success_possibility, _ in ddsim_result]
times_taken_aer = [time_taken  for _, _, time_taken in aer_result]
times_taken_ddsim = [time_taken  for _, _, time_taken in ddsim_result]

fig, axes = plt.subplots(ncols=2, figsize=(10,4))

# Success probability
axes[0].scatter(x, success_possibilities_aer, color="blue", s=20, label="AerSimulator")
axes[0].scatter(x, success_possibilities_ddsim, color="red", s=5, label="DDSIMulator")
axes[0].set_xlabel("the number of adders")
axes[0].set_ylabel("Avg. success probability")
axes[0].legend()

# Time taken
axes[1].scatter(x, times_taken_aer, color="blue", s=20, label="AerSimulator")
axes[1].scatter(x, times_taken_ddsim, color="red", s=5, label="DDSIMulator")
axes[1].set_xlabel("the number of adders")
axes[1].set_ylabel("Avg. time to execute")
axes[1].legend()

plt.savefig("ddsim_vs_aer_gates.png")

plt.show()

In [ ]:
%%time
random.seed(SEED)

num_state_qubits = 10
additional_max_depth = 50
num_trials = 10

additional_ddsim_jobs = dict()
for depth in range(max_depth + 1, additional_max_depth + 1):
    for _ in range(num_trials):
        max_number = (2 ** num_state_qubits) - 1
        a = random.randint(0, max_number)
        b = random.randint(0, max_number)
        
        # Get the target circuit.
        circuit = get_deep_circuit(num_state_qubits, a, b, depth)
        
        # Make the key for the dicts.
        result = (a * depth + b) % (2 ** (num_state_qubits + 1))
        key = (depth, num_trials, a, b, result)
    
        # Transpile the circuit for DDSIM.
        pass_manager = qiskit.transpiler.generate_preset_pass_manager(
            optimization_level=3,
            backend=ddsim_qasm,
            seed_transpiler=SEED,
        )
        trainspiled_circuit_for_ddsim = pass_manager.run(circuit)
        trainspiled_circuit_for_ddsim.add_register(
            qiskit.ClassicalRegister(circuit.num_qubits - circuit.num_state_qubits)
        )
        for c_index in range(trainspiled_circuit_for_ddsim.num_qubits - num_state_qubits):
            q_index = c_index + num_state_qubits
            trainspiled_circuit_for_ddsim.measure(q_index, c_index)
        # Run the circuit with DDSIM.
        job = ddsim_qasm.run(trainspiled_circuit_for_ddsim, shots=SHOTS)
        additional_ddsim_jobs[key] = job

In [ ]:
# Organise the data for DDSIM.
additional_ddsim_results = dict()
for key, ddsim_job in additional_ddsim_jobs.items():
    num_qubits = key[0]
    if num_qubits not in additional_ddsim_results:
        additional_ddsim_results[num_qubits] = {"success_possibilities": [], "time_taken": []}

    if not ddsim_job.result().results[0].success:
        continue
    
    true_key_digit = key[-1]
    additional_ddsim_results[num_qubits]["success_possibilities"].append(get_success_possibility(ddsim_job, true_key_digit))
    additional_ddsim_results[num_qubits]["time_taken"].append(get_time_taken_from_ddsim(ddsim_job))

additional_ddsim_result = [
    (num_qubits,
     float(np.mean(additional_ddsim_results[num_qubits]["success_possibilities"])),
     float(np.mean(additional_ddsim_results[num_qubits]["time_taken"])),
    )
    for num_qubits in additional_ddsim_results.keys()
]
additional_ddsim_result

In [ ]:
x = [num_qubits for num_qubits, _, _ in ddsim_result]
additional_x = [num_qubits for num_qubits, _, _ in additional_ddsim_result]

success_possibilities_ddsim = [success_possibility  for _, success_possibility, _ in ddsim_result]
times_taken_ddsim = [time_taken  for _, _, time_taken in ddsim_result]

additional_success_possibilities_ddsim = [success_possibility  for _, success_possibility, _ in additional_ddsim_result]
additional_times_taken_ddsim = [time_taken  for _, _, time_taken in additional_ddsim_result]

fig, axes = plt.subplots(ncols=2, figsize=(10,4))

# Success probability
axes[0].scatter(x, success_possibilities_ddsim, color="red", s=5, label="DDSIMulator")
axes[0].scatter(additional_x, additional_success_possibilities_ddsim, color="red", s=5, label="DDSIMulator")
axes[0].set_xlabel("the number of adders")
axes[0].set_ylabel("Avg. success probability")
axes[0].legend()

# Time taken
axes[1].scatter(x, times_taken_ddsim, color="red", s=5, label="DDSIMulator")
axes[1].scatter(additional_x, additional_times_taken_ddsim, color="red", s=5, label="DDSIMulator")
axes[1].set_xlabel("the number of adders")
axes[1].set_ylabel("Avg. time to execute")
axes[1].legend()

plt.savefig("ddsim_vs_aer_gates.png")

plt.show()